In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/3-2/"

/content/drive/MyDrive/Colab Notebooks/3-2


In [ ]:
#필요한 패키지 불러오기

import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

import yfinance as yf
yf.pdr_override()  # <-- Override to use yfinance when calling pandas_datareader

In [ ]:
#바이오 기업들 Id 불러와서 리스트로 저장
import pandas as pd

df = pd.read_parquet('./inner_bio_companyId.parquet')
indexes = list(df['companyId'])

In [ ]:
#바이오 마켓 데이터 불러오기
import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_market = pd.read_parquet('./market', filters=filters, columns=['companyId', 'pricingDate', 'priceClose', 'adjustmentFactor'])
df_market['companyId'] = list(map(int, df_market['companyId']))
df_market

In [ ]:
#바이오 예상치 데이터 불러오기
filters = [('companyId', 'in', indexes)]
df_estimates = pd.read_parquet('./estimates', filters=filters)

df_estimates['companyId'] = list(map(int, df_estimates['companyId']))
df_estimates['dataItemId'] = list(map(int, df_estimates['dataItemId']))
df_estimates['dataItemValue'] = list(map(float, df_estimates['dataItemValue']))
df_estimates['fiscalYear'] = list(map(int, df_estimates['fiscalYear']))

pd.options.mode.chained_assignment = None
df_est = df_estimates.copy()

#Estimates 데이터 단위 맞추는 작업

for i in range(len(df_est)):
  if df_est['estimateScaleName'][i] == 'Millions':
    df_est['dataItemValue'][i] = df_est['dataItemValue'][i] * 1_000_000
  elif df_est['estimateScaleName'][i] == 'Billions':
    df_est['dataItemValue'][i] = df_est['dataItemValue'][i] * 1_000_000_000
df_est.drop(['estimateScaleName'], axis=1, inplace=True)

#39414143 예상치 데이터가 -e^11이 나옴에 Outlier라고 판단하여 제거
df_est = df_est[df_est['companyId'] != 39414143]
df_est

In [ ]:
#환율 데이터 불러오기 + 21년 ~ 22년까지 Currency별로 데이터프레임 정리
import pyarrow.parquet as pq
import pandas as pd
df_exchange = pd.read_parquet('./exchange')

df_ex = df_exchange.copy()
df_ex.sort_values('priceDate', ascending=True, inplace=True)
df_ex.set_index('priceDate', drop=True, inplace=True)

code = list(df_ex['ISOCode'].unique())
fx = pd.DataFrame()

for c in code:
  ex = df_ex[(df_ex['ISOCode'] == c)].copy()
  ex.rename(columns={'priceClose' : c}, inplace=True)
  ex.drop(['ISOCode'], axis=1, inplace=True)
  d = ex.loc['2021-09-22':'2023-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  fx = pd.concat([fx, d], axis=1)

In [ ]:
#Estimates 데이터 항목 리스트
estimates_unique = df_est.groupby(['dataItemId','dataItemName', 'periodTypeName']).size().reset_index(name='freq')
estimates_unique.drop('freq', axis=1, inplace=True)

#예상치 데이터 중에 Non-periodic을 제외하고 Annual 데이터만 다룰거라 분류
annual = estimates_unique[estimates_unique['periodTypeName'] == 'Annual']
annual_item = list(annual['dataItemName'])
estimates_unique

In [ ]:
#마켓 데이터로부터 기업의 주가를 시계열로 데이터로 정리

df_price = df_market.copy()
df_price['price'] = df_price['priceClose'] * df_price['adjustmentFactor']
df_price['price'] = df_price['price'].apply(lambda x: round(x, 4))
df_price.drop(['priceClose', 'adjustmentFactor'], axis=1, inplace=True)

df_price['companyId'] = list(map(int, df_price['companyId']))
unique_id = list(df_price['companyId'].unique())
market = pd.DataFrame()

In [ ]:
#마켓 데이터로부터 기업의 주가를 시계열로 데이터로 정리

df_price = df_market.copy()
df_price['price'] = df_price['priceClose'] * df_price['adjustmentFactor']
df_price['price'] = df_price['price'].apply(lambda x: round(x, 4))
df_price.drop(['priceClose', 'adjustmentFactor'], axis=1, inplace=True)

df_price['companyId'] = list(map(int, df_price['companyId']))
unique_id = list(df_price['companyId'].unique())
market = pd.DataFrame()

#팩터 모델링을 위한 Training의 기간은 21년 9월 22일부터 22년 9월 22일까지
for id in unique_id:
  ex = df_price[(df_price['companyId'] == id)].copy()
  ex.rename(columns={'price' : id}, inplace=True)
  ex.drop(['companyId'], axis=1, inplace=True)
  ex.sort_values(by='pricingDate', inplace=True)
  ex.set_index('pricingDate', inplace=True, drop=True)
  d = ex.loc['2021-09-22':'2022-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  market = pd.concat([market, d], axis=1)

market2 = market.copy()
mark_date = market2.isnull().sum(axis=1).index
mark_col = market2.isnull().sum(axis=0).index

#한 날짜에 850개 초과의 기업이 주가가 없다면 해당 날짜 제거 (공휴일이라 판단)
for i in mark_date:
  if market2.isnull().sum(axis=1)[i] > 850:
    market2.drop(i, inplace=True)

#한 기업이 40일 이상의 주가가 없으면 제거 (결측치가 너무 많으면 평균이나 선형보간법으로 결측치를 채우는게 무리가 있다고 판단)
for i in mark_col:
  if market2.isnull().sum(axis=0)[i] > 40:
    market2.drop(i, axis=1, inplace=True)

#선형 보간법으로 우선 채우고 연속된 결측치의 경우에는 전 값으로 결측치를 처리
market2.interpolate(method='linear', inplace=True)
market2.fillna(method='bfill', inplace=True)
market2.drop(39414143, axis=1, inplace=True)

In [ ]:
#Estimates 데이터 중에 ISOCode가 없는 데이터들이 존재하는데 이를 제거

for a in annual_item:
  d = df_est[(df_est['dataItemName'] == a)]['ISOCode'].isnull().sum()
  if d > 0:
    annual_item.remove(a)

len(annual_item)

In [ ]:
#한 예상치 데이터에 대해 크기 순으로 정렬하여 상위 50%, 하위 50%를 기준으로 롱숏 포트폴리오를 구성 후 수익률 산출
#예상치 데이터는 연간으로 나타나기 때문에 21년을 데이터를 바탕으로 21년 9월 22일 ~ 21년 12월 31일까지의 포트폴리오를 구성
#22년 데이터에 대해서는 22년 1월 1일~22년 9월 22일까지의 포트폴리오를 구성

def calculate(item, year):
  market3 = market2.copy()
  #아이템이랑 연도 받아서 데이터프레임 생성
  example_feature = df_est[(df_est['dataItemName'] == item)  & (df_est['fiscalYear'] == year)]
  example = example_feature[['companyId', 'dataItemValue', 'ISOCode']].copy()

  #같은 시점에 똑같은 기업이 똑같은 예상치 데이터에 대해 다른 값을 가지고 있다면 평균 내고 하나의 값으로 변환
  #ex) 21년에 A 기업의 EBITDA Estimate이 다른 값으로 2~3개가 있음 -> 평균 내서 하나만 남기기

  example2 = example.copy()
  for id in list(map(int, example2['companyId'])):
    num = len(example2[example2['companyId']==id]['dataItemValue'].values)
    if num > 1:
      idx = list(example2[example2['companyId']==id]['dataItemValue'].index)
      avg = []
      for i in idx:
        a = example2.loc[i]['dataItemValue']
        avg.append(a)
      average = sum(avg) / len(avg)
      example2.loc[idx[0]]['dataItemValue'] = average
      example2.drop([i for i in idx[0:]], inplace=True)
  example2.reset_index(drop=True, inplace=True)

  #예상치 데이터가 어느 나라의 기업이냐에 따라 명시되어 있는 Currency가 다름 (ISOCode를 통해 확인 가능)
  #21년 예상치 데이터를 활용하여 21년 9월 22일부터 ~ 21년 말까지의 포트폴리오를 구성하기 때문에,
  #달러로 표시되어 있는게 아니라면 21년 9월 22일의 환율을 기준으로 값을 달러로 변경
  #22년은 1월 1일 환율을 기준으로 값을 달러로 변경

  if year == 2021:
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2021-09-22", example2['ISOCode'][i]])
  else:
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-01-01", example2['ISOCode'][i]])

  example2.drop('ISOCode', axis=1, inplace=True)
  example2.sort_values(by='dataItemValue', ascending=False, inplace=True)

  #데이터가 있는 기업들의 주가를 불러옴
  comp_id = list(map(int, example2['companyId']))
  market3 = market3.reindex(columns=comp_id)
  market3 = market3.dropna(axis=1)

  #년도에 따라서 포트폴리오를 구성하는 기간이 다르기 때문에 그 구간에서의 주가를 가져옴
  if year == 2021:
    price_y = market3.loc['2021-09-22':'2021-12-31'].copy()
  elif year == 2022:
    price_y = market3.loc['2022-01-01':'2022-09-22'].copy()

  for col in price_y.columns:
    price_y[col] = price_y[col].apply(lambda x: float(x))

  #주가를 수익률로 변환 후에 크기 순으로 정렬되어 있기 때문에 상위 50퍼는 롱,
  #하위 50퍼는 숏을 갖는 포트폴리오의 수익률을 새로운 칼럼으로 생성
  price_y = price_y.pct_change(1).dropna()
  price_y = price_y + 1
  cum_ret = price_y.cumprod() - 1
  half = int(len(cum_ret.columns) / 2)
  cum_ret[item] = (cum_ret.iloc[:, 0:half+1].sum(axis=1) - cum_ret.iloc[:, half+1:].sum(axis=1)) / len(cum_ret.columns)
  return cum_ret[item]

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import statsmodels.api as sm

#21년 9월 22일부터 ~ 22년 9월 22일까지 각 예상치 데이터에 대한 롱숏 포트폴리오를 구성 후 일별 수익률을 계산

pd.options.mode.chained_assignment = None
year = [2022]
multi_agg = pd.DataFrame()
for item in annual_item:
  pf = calculate(item, 2021)
  for y in year:
    temp = calculate(item, y)
    pf = pd.concat([pf, temp], axis=0)
  multi_agg = pd.concat([multi_agg, pf], axis=1)
multi_agg

In [ ]:
#데이터프레임 생성 과정에서 결측치 존재하면 칼럼 제거
port = multi_agg.copy()
columns_with_missing_values = list(port.columns[port.isnull().any()])
port.drop(columns_with_missing_values, axis=1, inplace=True)

In [ ]:
#주가 데이터로부터 수익률을 구하고 첫쨰날을 제거
market4 = market2.copy()
market4 = market4.pct_change(1)
market4.drop('2021-09-22', inplace=True)

#주가 수익률과 포트폴리오 수익률을 합쳐주며 특정 날짜에 Null 값이 존재하면 드랍
estimates_final = pd.concat([market4, port], axis=1)
a = estimates_final.index[estimates_final.isnull().any(axis=1)]
estimates_final.drop(a, inplace=True)
estimates_final

In [ ]:
port.corr()

In [ ]:
#Total은 바이오 섹터에 속하는 개별 기업의 개수
total = len(estimates_final.columns[:-24])
#Port_col은 각 포트폴리오의 이름을 의미
port_col = list(port.columns)

#개별 기업의 수익률을 y로 두고, 각 데이터에 대한 포트폴리오를 X로 두고 1158개에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
for p in port_col:
  cnt = 0
  for col in estimates_final.columns[:-24]:
    res = sm.OLS(endog=estimates_final[col], exog=sm.add_constant(estimates_final[p])).fit()-
    if res.pvalues[1] <= 0.05:
      cnt += 1
  print(p, cnt)